In [76]:
import mtt
from os.path import join, realpath
from os import getcwd
sbfile = join(realpath(getcwd()),'catalysis.sb')
wiring = mtt.Wiring.fromFile2(sbfile)
wiring

In [77]:
model = mtt.MTT(wiring)
model.draw(3)

# Shift Register Alterations

We need to manually set Btot and KDfw on the last block if we want to match Sung's example.

In [78]:
wiring.blocks[0].KDrv = 50.

wiring.blocks[2].Btot = 100.
wiring.blocks[2].KDfw = 100.
wiring.blocks[2].kr = 5.
wiring.blocks[2].KDrv = 10. # not used

# Digitized current config

In [79]:
for block in wiring.blocks:
    print(block.getDigitizedParameterString(model))

# Generate SRAM Program

In [80]:
from mtt import addSRAMRule, removeSRAMRule
program = model.compileSRAM()
rules = program.getRoutingRulesBinaryMatrix()

# ADC connections
# group, block, variable, wire
# variables:
Ctot_var  = 11
Afree_var = 12
# rules = addSRAMRule(rules,0,1,Afree_var,88) # "A"
# rules = addSRAMRule(rules,0,2,Afree_var,89) # "B"
# rules = addSRAMRule(rules,0,2,Ctot_var,90)  # "C"

# rules = addSRAMRule(rules,0,0,Ctot_var,91) # Ctot for first  block (total A)
# rules = addSRAMRule(rules,0,1,Ctot_var,92) # Ctot for second block (total B)

# five extra ADC connections from group 2 block 1
# rules = addSRAMRule(rules,2,1,18,94)
# rules = addSRAMRule(rules,2,1,19,95)
# rules = addSRAMRule(rules,2,1,12,96)
# rules = addSRAMRule(rules,2,1,7,97)
# rules = addSRAMRule(rules,2,1,25,98)

# wire to ADC
# for i in range(88,93+1):
#     rules = addSRAMRule(rules,0,0,31,i)

# Sung used port 10 for Ctot instead of port 9, both are valid
rules = removeSRAMRule(rules,0,1,9)
rules = addSRAMRule(rules,0,1,10,3)

In [81]:
import pprint
pp = pprint.PrettyPrinter(indent=2, depth=1, width=10)
pp.pprint(program.dump())

# Routing Configuration

One row per wire.

In [82]:
from numpy import array
binrules = array([bin(rule[0])[2:].zfill(10) for rule in rules])
print(binrules.reshape((-1,1)))

# SRAM Layout

Rows are binary routing config rules, cols are wires.

In [83]:
from scipy.io import savemat
sram_matrix = program.convertToSRAMMatrix(rules)
savemat('sram.mat', {'in_all': sram_matrix})

# Shift Registers

In [84]:
def sr_alterations(prog_vect):
    # sung configures one of the registers to have unity current of 12 nA
    prog_vect = array(prog_vect)
    prog_vect[280:287,0] = [0, 1, 0, 1, 0, 1, 0]
    return prog_vect

In [85]:
sr = model.compileShiftReg()
prog_vect = sr_alterations(sr.getProgrammingVector(use_Ctot=False))
savemat('sr1.mat', {'prog_vect': prog_vect})

# Simulink Export

In [86]:
model.exportToSimulink(join(realpath(getcwd()),'simulink'), 'cascade', 3)

# Initial Conditions

Shift register configuration for initial conditions

In [87]:
wiring.blocks[0].Atot = 0.
wiring.blocks[1].Atot = 100.
sr = model.compileShiftReg()
prog_vect = sr_alterations(sr.getProgrammingVector(use_Ctot=True))
savemat('sr2.mat', {'prog_vect': prog_vect})

In [88]:
# import numpy
# numpy.set_printoptions(threshold=numpy.nan)
# print(prog_vect)

# Simulation

In [89]:
# hardware simulation
model.reset()
wiring.blocks[0].Atot = 0.
wiring.blocks[1].Atot = 100.
# for k in range(3):
#     print('block {}: rate_fw = {}'.format(k,model.blocks[k].rate_fw))
#     print('block {}: rate = {}'.format(k,model.blocks[k].rate))
r = model.simulate(0,10,100)
# print(r)
model.plot()

# SBML Simulation

In [90]:
# SBML comparison
import tellurium as te
with open(join(realpath(getcwd()),'catalysis.sb')) as f:
    rr_model = te.loada(f.read())
print(rr_model.getReactionRates())
rr_model.simulate(0,10,1000)
rr_model.plot()